In [99]:
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

## 데이터 업로드

In [100]:
kospi31 = pd.read_csv("./data/preprocessed_kospi31_daily_stock_chart.csv")
kospi31

,code,name,date,open,high,low,close,z_d5,z_d10,z_d15,z_d20,z_d25,z_d30
0,U182,kospi50_index,20030102,-0.000572,0.003749,-0.017282,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,U182,kospi50_index,20030103,-0.016228,0.002173,-0.020027,0.044230,-0.021178,-0.021178,-0.021178,-0.021178,-0.021178,-0.021178
2,U182,kospi50_index,20030106,-0.003413,0.002105,-0.014164,0.009321,-0.020145,-0.020145,-0.020145,-0.020145,-0.020145,-0.020145
3,U182,kospi50_index,20030107,0.042358,0.042912,-0.000457,-0.022060,0.001468,0.001468,0.001468,0.001468,0.001468,0.001468
4,U182,kospi50_index,20030108,0.007934,0.014432,-0.001829,-0.000962,0.001946,0.001946,0.001946,0.001946,0.001946,0.001946
...,...,...,...,...,...,...,...,...,...,...,...,...,...
147835,A005380,현대차,20210826,-0.002381,0.009524,-0.009524,0.002387,-0.014762,0.001190,0.017937,0.027381,0.036381,0.045238
147836,A005380,현대차,20210827,0.002398,0.004796,-0.016787,-0.007143,-0.000959,0.003357,0.020624,0.031535,0.040000,0.049081
147837,A005380,현대차,20210830,0.004762,0.011905,0.000000,0.007194,-0.002381,-0.007143,0.009365,0.022262,0.029619,0.038413
147838,A005380,현대차,20210831,-0.011765,0.000000,-0.021176,0.011905,-0.011294,-0.019059,-0.005176,0.008471,0.015247,0.023843


## PreProcessing - raw 데이터 입력형태 맞추기

#### Market index

In [105]:
market_index = kospi31[kospi31.code == "U182"]
market_index

,code,name,date,open,high,low,close,z_d5,z_d10,z_d15,z_d20,z_d25,z_d30
0,U182,kospi50_index,20030102,-0.000572,0.003749,-0.017282,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,U182,kospi50_index,20030103,-0.016228,0.002173,-0.020027,0.044230,-0.021178,-0.021178,-0.021178,-0.021178,-0.021178,-0.021178
2,U182,kospi50_index,20030106,-0.003413,0.002105,-0.014164,0.009321,-0.020145,-0.020145,-0.020145,-0.020145,-0.020145,-0.020145
3,U182,kospi50_index,20030107,0.042358,0.042912,-0.000457,-0.022060,0.001468,0.001468,0.001468,0.001468,0.001468,0.001468
4,U182,kospi50_index,20030108,0.007934,0.014432,-0.001829,-0.000962,0.001946,0.001946,0.001946,0.001946,0.001946,0.001946
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4615,U182,kospi50_index,20210826,0.010887,0.013703,-0.001595,-0.009858,-0.001292,0.005035,0.020108,0.026727,0.030682,0.033961
4616,U182,kospi50_index,20210827,-0.003189,0.003255,-0.008329,-0.001550,0.003172,0.003445,0.017568,0.025910,0.030196,0.033372
4617,U182,kospi50_index,20210830,0.007979,0.009285,-0.003514,0.001191,0.003508,0.000603,0.012611,0.023071,0.026918,0.030145
4618,U182,kospi50_index,20210831,-0.020602,0.000322,-0.025145,0.021590,-0.015049,-0.019113,-0.010892,0.000475,0.004420,0.007533


In [106]:
stocks = kospi31[kospi31.code != "U182"]
stocks

,code,name,date,open,high,low,close,z_d5,z_d10,z_d15,z_d20,z_d25,z_d30
4620,A030200,KT,20030102,0.018036,0.022044,-0.006012,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4621,A030200,KT,20030103,-0.034351,0.000000,-0.041985,0.050100,-0.023855,-0.023855,-0.023855,-0.023855,-0.023855,-0.023855
4622,A030200,KT,20030106,-0.009542,0.005725,-0.019084,0.000000,-0.015903,-0.015903,-0.015903,-0.015903,-0.015903,-0.015903
4623,A030200,KT,20030107,0.036893,0.036893,-0.003883,-0.017176,0.000971,0.000971,0.000971,0.000971,0.000971,0.000971
4624,A030200,KT,20030108,0.003846,0.019231,-0.007692,0.009709,-0.006923,-0.006923,-0.006923,-0.006923,-0.006923,-0.006923
...,...,...,...,...,...,...,...,...,...,...,...,...,...
147835,A005380,현대차,20210826,-0.002381,0.009524,-0.009524,0.002387,-0.014762,0.001190,0.017937,0.027381,0.036381,0.045238
147836,A005380,현대차,20210827,0.002398,0.004796,-0.016787,-0.007143,-0.000959,0.003357,0.020624,0.031535,0.040000,0.049081
147837,A005380,현대차,20210830,0.004762,0.011905,0.000000,0.007194,-0.002381,-0.007143,0.009365,0.022262,0.029619,0.038413
147838,A005380,현대차,20210831,-0.011765,0.000000,-0.021176,0.011905,-0.011294,-0.019059,-0.005176,0.008471,0.015247,0.023843


In [124]:
def train_vaild_test_split(df):
    codes = df.code.unique()
    train_data = pd.DataFrame()
    vaild_data = pd.DataFrame()
    test_data = pd.DataFrame()
    for c in codes:
        df_stock = df[df.code == c]
        df_stock.reset_index(drop=True, inplace=True)
        times = sorted(df_stock.index.values)
        last_10pct = sorted(df_stock.index.values)[-int(0.1*len(times))] # Last 10% of series
        last_20pct = sorted(df_stock.index.values)[-int(0.2*len(times))] # Last 20% of series
        
        df_train = df_stock[(df_stock.index < last_20pct)]  # Training data are 80% of total data
        df_val = df_stock[(df_stock.index >= last_20pct) & (df_stock.index < last_10pct)]
        df_test = df_stock[(df_stock.index >= last_10pct)]

        train_data = pd.concat([train_data, df_train])
        vaild_data = pd.concat([vaild_data, df_val])
        test_data = pd.concat([test_data, df_test])
    return train_data, vaild_data, test_data

In [125]:
market_train_data, market_vaild_data, market_test_data = train_vaild_test_split(market_index)
stock_train_data, stock_vaild_data, stock_test_data = train_vaild_test_split(stocks)
print(market_train_data.shape, market_vaild_data.shape, market_test_data.shape)
print(stock_train_data.shape, stock_vaild_data.shape, stock_test_data.shape)

(3696, 13) (462, 13) (462, 13)
(114576, 13) (14322, 13) (14322, 13)


In [126]:
market_train_data

,code,name,date,open,high,low,close,z_d5,z_d10,z_d15,z_d20,z_d25,z_d30
0,U182,kospi50_index,20030102,-0.000572,0.003749,-0.017282,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,U182,kospi50_index,20030103,-0.016228,0.002173,-0.020027,0.044230,-0.021178,-0.021178,-0.021178,-0.021178,-0.021178,-0.021178
2,U182,kospi50_index,20030106,-0.003413,0.002105,-0.014164,0.009321,-0.020145,-0.020145,-0.020145,-0.020145,-0.020145,-0.020145
3,U182,kospi50_index,20030107,0.042358,0.042912,-0.000457,-0.022060,0.001468,0.001468,0.001468,0.001468,0.001468,0.001468
4,U182,kospi50_index,20030108,0.007934,0.014432,-0.001829,-0.000962,0.001946,0.001946,0.001946,0.001946,0.001946,0.001946
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3691,U182,kospi50_index,20171124,-0.002272,0.000000,-0.005023,0.002754,-0.002788,-0.002757,0.000688,0.001940,-0.002313,-0.005487
3692,U182,kospi50_index,20171127,0.020006,0.021236,-0.000027,-0.021229,0.016019,0.016693,0.020255,0.023266,0.019118,0.016096
3693,U182,kospi50_index,20171128,-0.002886,0.000929,-0.008966,0.005046,0.008701,0.010240,0.013598,0.017400,0.014097,0.011092
3694,U182,kospi50_index,20171129,0.002690,0.004493,-0.001956,-0.001809,0.006520,0.011033,0.013533,0.017503,0.015920,0.012971


In [127]:
market_vaild_data

,code,name,date,open,high,low,close,z_d5,z_d10,z_d15,z_d20,z_d25,z_d30
3696,U182,kospi50_index,20171201,0.006770,0.007652,-0.000269,-0.001797,0.012567,0.024638,0.028683,0.033384,0.035814,0.033097
3697,U182,kospi50_index,20171204,-0.009167,0.000000,-0.013349,0.013655,-0.001840,0.009036,0.013112,0.017663,0.021381,0.018879
3698,U182,kospi50_index,20171205,-0.007694,0.002055,-0.009194,-0.000032,-0.003594,0.007057,0.011642,0.016087,0.020468,0.018694
3699,U182,kospi50_index,20171206,0.013078,0.015418,-0.000046,-0.013697,0.006025,0.016886,0.023578,0.027687,0.032374,0.032124
3700,U182,kospi50_index,20171207,0.003064,0.004533,-0.003677,-0.000887,0.006328,0.013994,0.021840,0.026180,0.031127,0.032572
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4153,U182,kospi50_index,20191017,0.000438,0.003533,-0.003806,-0.002607,-0.004134,-0.014510,-0.012394,-0.009903,-0.012541,-0.019412
4154,U182,kospi50_index,20191018,0.012190,0.016704,-0.001757,-0.011156,0.007639,-0.001750,-0.001640,0.001439,-0.000620,-0.006563
4155,U182,kospi50_index,20191021,-0.000667,0.001329,-0.004472,0.004438,0.002762,-0.003812,-0.006270,-0.003044,-0.004239,-0.008900
4156,U182,kospi50_index,20191022,-0.007862,0.002867,-0.009460,0.013778,-0.008537,-0.013804,-0.018202,-0.016241,-0.016705,-0.020312


In [128]:
market_test_data

,code,name,date,open,high,low,close,z_d5,z_d10,z_d15,z_d20,z_d25,z_d30
4158,U182,kospi50_index,20191024,0.003876,0.004190,-0.008156,0.002652,-0.006692,-0.008713,-0.016258,-0.015877,-0.014510,-0.017020
4159,U182,kospi50_index,20191025,0.002296,0.002678,-0.004889,0.001343,-0.004219,-0.007877,-0.015236,-0.016532,-0.014937,-0.017066
4160,U182,kospi50_index,20191028,0.000941,0.005569,-0.001798,0.002747,-0.003472,-0.009066,-0.015237,-0.018519,-0.016986,-0.018527
4161,U182,kospi50_index,20191029,0.005506,0.006395,-0.003147,-0.000132,-0.002600,-0.007404,-0.012503,-0.017028,-0.016427,-0.017388
4162,U182,kospi50_index,20191030,0.006428,0.006428,-0.002359,-0.006887,0.004254,-0.000413,-0.004425,-0.009572,-0.009510,-0.009839
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4615,U182,kospi50_index,20210826,0.010887,0.013703,-0.001595,-0.009858,-0.001292,0.005035,0.020108,0.026727,0.030682,0.033961
4616,U182,kospi50_index,20210827,-0.003189,0.003255,-0.008329,-0.001550,0.003172,0.003445,0.017568,0.025910,0.030196,0.033372
4617,U182,kospi50_index,20210830,0.007979,0.009285,-0.003514,0.001191,0.003508,0.000603,0.012611,0.023071,0.026918,0.030145
4618,U182,kospi50_index,20210831,-0.020602,0.000322,-0.025145,0.021590,-0.015049,-0.019113,-0.010892,0.000475,0.004420,0.007533


In [174]:
def get_input_data(df):
    codes = df.code.unique()
    X_data, y_data = [], []
    for c in codes:
        df_stock = df[df.code == c]
        df_stock.drop(columns=['code'], inplace=True)
        df_stock.drop(columns=['name'], inplace=True)
        df_stock.drop(columns=['date'], inplace=True)
        
        nums_features = len(df_stock.columns)
        df_stock = df_stock.values
        
        X, y = [], []
        seq_len = 10
        for i in range(seq_len, len(df_stock)):
            X.append(df_stock[i-seq_len:i])
            y.append(df_stock[:, 3][i])
        X, y = np.array(X), np.array(y)
        X_data.append(X)
        y_data.append(y)
    X_data = np.array(X_data)
    y_data = np.array(y_data)
    X_data = X_data.reshape(-1, len(codes), seq_len, nums_features)
    y_data = y_data.reshape(-1, len(codes))
    print(X_data.shape, y_data.shape)
#     X_data = np.concatenate((X_data, X), axis=1)
    return X_data, y_data

In [176]:
market_train_X, _ = get_input_data(market_train_data)
market_vaild_X, _ = get_input_data(market_vaild_data)
market_test_X, _ = get_input_data(market_test_data)

stock_train_X, stock_train_y = get_input_data(stock_train_data)
stock_vaild_X, stock_vaild_y = get_input_data(stock_vaild_data)
stock_test_X, stock_test_y = get_input_data(stock_test_data)

(3686, 1, 10, 10) (3686, 1)
(452, 1, 10, 10) (452, 1)
(452, 1, 10, 10) (452, 1)
(3686, 31, 10, 10) (3686, 31)
(452, 31, 10, 10) (452, 31)
(452, 31, 10, 10) (452, 31)


### Model
